Unfortunately Github doesn't render ipywidgets, which allow us to interactively alter the parameters of the model. If you want to mess with the parameters yourself then download this file, and also install and activate ipywidgets, which you can do in the command line interface with the following commands:
```
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

In [3]:
#dependencies
import numpy as np
import operator as op
from functools import reduce
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import cmath

Parameters:
<br>
$\lambda$ = probability of encountering resources <br>
$S(t)$ = number of solitary locusts at time t [locusts] <br>
$G(t)$ = number of gregarious locusts at time t [locusts] <br>
$R(t)$ = number of resources at time t [mass] <br>
$R^*$ = Value of resources for which consumption rate saturates [mass] <br>
$R_c$ = carrying capacity for resources [mass] <br>
$\alpha$ = percentage of total resources found in a patch, proxy for clumpiness <br>
$k$ = proportion of resources reserved for a solitary locust before gregarious locusts arrive <br>
$h_0$ = foraging rate [mass/time] <br>
$\mu$ = energy cost [mass/time] <br> 
$r$ = resource regeneration rate [1/time] <br>
$p_s, p_g$ = proportion of solitary, gregarious locusts <br>
Note that $G(t)$ and $S(t)$ actually stay the same, but the energy of locusts' changes

<h3>The model:</h3><br>
<h4>Change in energy for gregarious locusts</h4><br>
$$
G(t)\frac{dE_G}{dt} = \left [\sum_{i=0}^{S(t)}{S(t) \choose i}\lambda_s^i(1-\lambda_s)^{S(t)-i} i\right ](1-p_sk) \alpha p_sh(R) + \lambda_g \alpha p_gh(R)G(t) - \mu_GG(t)
$$
Where $h(R)=h_0\frac{R(t)}{(R(t)+R^*)}$ (Instead of $R(t)$ this could be the expected number of resources)
<p>Gregarious locusts gain energy both by discovering resources themselves (second term on the right) and by scrounging off of solitary locusts (first term)</p>


<h4>Change in energy for solitary locusts</h4>
<br>
$$
S(t)\frac{dE_S}{dt} = \left [\sum_{i=0}^{S(t)} {S(t) \choose i} \lambda_s^i(1-\lambda_s)^{S(t)-i}i \right ]p_sk \alpha h(R) - \mu_SS(t)
$$

<h4>Change in resources</h4>
$$
\frac{dR}{dt} = -(S(t)(\frac{dE_S}{dt}+\mu_s) + G(t)(\frac{dE_G}{dt}+\mu_G))+rR(t)(1-R(t)/R_c)
$$

In [4]:
#a helper function to compute n choose k
def choose(n,r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

In [5]:
##implementation of dE_G/dt
def DE(S, L, R, alpha, k, G, h, us, ug, r, Rn, Rc):
    glamb = G/L
    slamb = S/L
    ps=S/(S+G)
    pg=G/(S+G)
    
    #dE_G/dt
    x=0
    for i in range(S):
        x += choose(S,i)*(slamb**i)*((1-slamb)**(S-i))*i
    h = h*R/(R+Rn)
    x=x*h*ps*(1-ps*k)*alpha
    x += glamb*G*alpha*pg*h - ug*G
    
    #dE_S/dt
    y = 0
    for i in range(S):
        y += choose(S,i)*((slamb**i)*((1-slamb)**(S-i))*i)
    y = y*ps*k*alpha*h - us*S
    
    #dR/dt
    z = r*R*(1-(R/Rc)) - S*(y+us) - G*(x+ug)
    return x, y, z

In [6]:
@interact(k=(.05,1,.1), h=(.05, .5, .05), alpha=(0,1,.05), us=(.001,.015, .001), ug=(.001,.015, .001))
def simulate(S=(10,50), L=100, G=(10,50), Rn=30, alpha=.3, k=.35, h=.05, us=.004, ug=.007, r=.2, Rc=100, tsteps=10000):
    dt = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [0,0,Rn]
    for i in range(tsteps):
        g_dot, s_dot, R_dot = DE(S, L, x[i][2], alpha, k, G, h, us, ug, r, Rn, Rc)
        x[i+1][0] = x[i][0]+G*g_dot*dt
        x[i+1][1] = x[i][1]+S*s_dot*dt
        x[i+1][2] = x[i][2]+R_dot*dt
    g = [y[0] for y in x]
    s = [y[1] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=30, description='S', max=50, min=10), IntSlider(value=100, description='…

<h3> Non Dimensionalization </h3>
<br>
We would like to make the system of equations dimensionless so that we can deal with fewer parameters <br>
First note that the sums in the first two DEs evaluate to the expected value of a binomial distribution, which in this case would be $S(t) \lambda_s$ 
<br>
We set up a time scale $\tau = t/T$ for $T=1/\mu_s$. Thus $\frac{dE_S}{dt}=\frac{dE_S}{d\tau}\frac{d\tau}{dt}=\mu_s\frac{dE_S}{d\tau}$
<br>
Thus our DE for solitary locusts' energy becomes:
$$
\mu_s S(t) \frac{dE_S}{d\tau} = S(t)\lambda_s p_sk \alpha h(R) - \mu_sS(t) = S(t)\lambda_s k \alpha h_0 p_s \frac{R(t)}{R(t)+R^*} - \mu_sS(t) 
$$
<br>
Dividing through by $\mu_s S(t)$ yields
$$
\frac{dE_S}{d\tau} = S(t) k \gamma p_s \frac{R(t)}{R(t)+R^*} - 1 
$$

where $\gamma = \frac{\alpha h_0}{\mu_s L}$
<br>
<br>
We also apply this non dimensionalization to gregarious locusts' energy
$$
\mu_sG(t)\frac{dE_G}{d\tau} = S(t)\lambda_s\alpha p_s h(R) - S(t)\lambda_s\alpha kp_s^2h(R) + \lambda_g \alpha p_gh(R)G(t) - \mu_GG(t)
$$
Dividing through yields:
$$
\frac{dE_G}{d\tau} = S(t)q(1-kp_s)\gamma p_s\frac{R(t)}{R(t)+R^*} + G(t)\gamma p_g \frac{R(t)}{R(t)+R^*} - \mu_l
$$
where $\mu_l = \mu_g/\mu_s$ and $q=S(t)/G(t)$
<br>
<br>
Finally, non dimensionalizing the resource DE yields:
$$
\frac{dR}{d\tau} = r^*R(t)(1-R(t)/R_c) - S(t)(\frac{dE_s}{d\tau}+1) - G(t)(\frac{dE_g}{d\tau}+\mu_l)
$$
where $r^*=r/\mu_s$

In [7]:
def altDE(S, R, k, G, ul, r, Rn, gamma, Rc):
    q = S/G
    ps = S/(S+G)
    pg = G/(S+G)
    h = R/(R+Rn)
    
    
    #dE_G/dtau
    x = S*q*(1-k*ps)*gamma*ps*h + G*gamma*pg*h - ul
    
    #dE_S/dt
    y = S*k*gamma*ps*h - 1
    
    #dR/dt
    z = r*R*(1-(R/Rc)) - S*(y+1) - G*(x+ul)
    return x, y, z


In [8]:
@interact(k=(.05,1.5,.05), S=(0,50,1), G=(0,50,1), ul=(1,3,.1), gamma=(.01,5,.01), Rn=(0,2000,10), Rc=(0,2000), tsteps=(0,10000,100))
def simulate(S=15, G=15, Rn=30, k=.35, ul=1.5, r=.2, gamma=.9, Rc=100, R0=40, tsteps=1000):
    q = S/G
    ps = S/(S+G)
    pg = G/(S+G)
    
    beta = (q**2)*(1-k*ps)*ps + pg
    kappa = (S**2)*k*ps + (G**2)*beta
    coeff = [-r/Rc, r*(1-Rn/Rc), r*Rn-gamma*kappa]
    print(np.roots(coeff))
    gcrit = (1/kappa)*(r*Rn+(Rc*r/4)*(1-Rn/Rc)**2)
    print(gcrit)
    
    dtau = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [0,0,R0]
    for i in range(tsteps):
        g_dot, s_dot, R_dot = altDE(S, x[i][2], k, G, ul, r, Rn, gamma, Rc)
        x[i+1][0] = x[i][0]+G*g_dot*dtau
        x[i+1][1] = x[i][1]+S*s_dot*dtau
        x[i+1][2] = x[i][2]+R_dot*dtau
    g = [y[0] for y in x]
    s = [y[1] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=15, description='S', max=50), IntSlider(value=15, description='G', max=5…

If you change k to the critical value and then change gamma such that the nullclines are positive, you should see that the nullclines are pretty close by.

<h3> Equilibrium for R(t) </h3>
We set $dR/d\tau=0$ to find an equilibrium
$$
0 = \frac{dR}{d\tau} = r^*R(t)(1-R(t)/R_c) - \frac{R(t)}{R(t)+R^*}(S^2(t)k\gamma p_s +G^2(t)\gamma \beta)
$$
Rearranging yields:
$$
r^*(R(t)+R^*)(1-R(t)/R_c)=\gamma \kappa
$$
Where $\kappa=S^2(t)kp_s + G^2(t)\beta$ <br>
Thus, we get the following solution for $R(t)$ via the quadratic equation:
$$
R(t) = \frac{-r^*(1 - \frac{R^*}{R_c}) \pm \sqrt{(r^*(1-\frac{R^*}{R_c}))^2 + 4(\frac{r^*}{R_c}(r^*R^*-\gamma\kappa))}}{2r^*/R_c}
$$
This has a real solution when the terms in the square root are positive, which requires
$$
\gamma \leq \frac{1}{\kappa}(r^* R^* + \frac{R_c r^*}{4}(1-\frac{R^*}{R_c})^2)
$$
We then conduct a stability analysis of this fixed point by taking the derivate of $dR/d\tau$ relative to $R$:
$$
f'(R) = r^* - \frac{R^*}{(R+R^*)^2}\kappa \gamma
$$


<h3>Adding a genetic component</h3> <br>
Let's say that the population of locusts increases and decreases relative to the locusts' energy. So for the solitary locusts:
$$
\frac{dS}{dt}= E_sgS
$$
where $\left[ g \right]=1/(mass*time)$ and $g << h_0$ (although they don't have the same units), the consumption rate. In Einsteinian fashion, we can equate one locust with a certain amount of energy, such that $S=E_s/C$. Thus we can rearrange the above DE:
$$
\frac{dE_s}{dt} = CE_sgS = E_S^2g
$$
So we can add this component to our original DE for energy:
$$
S(t)\frac{dE_S}{dt} = S(t)\lambda p_sk \alpha h(R) - \mu_SS(t) + CE_sgS^2(t)
$$
Non dimensionalizing yields
$$
\frac{dE_S}{d\tau} = S(t) k \gamma p_s \frac{R(t)}{R(t)+R^*} - 1 + \frac{E_Sg}{\mu_s}S(t)
$$
$$
= \gamma \zeta E_s \frac{R(t)}{R(t)+R^*} - 1 + \rho E_S^2
$$
where $\rho = g/\mu_s$ and $\zeta = kp_s/C$
<br>
<br>
Similarily, for gregarized locusts, adding a genetic component would yield:
$$
\frac{dE_G}{d\tau} = S(t)q(1-kp_s)\gamma p_s\frac{R(t)}{R(t)+R^*} + G(t)\gamma p_g \frac{R(t)}{R(t)+R^*} - \mu_l + \rho E_G^2
$$
Just to clean this up,
$$
\frac{dE_G}{d\tau} = \gamma \omega E_G \frac{R(t)}{R(t)+R^*} - \mu_l + \rho E_G^2
$$
where $\omega = (q^2(1-kp_s)p_s+p_g)/C$
<br>
The change in resources would still only take into account consumption of gregarious and solitary locusts
$$
\frac{dR}{d\tau} = r^*R(t)(1-\frac{R(t)}{R_c}) - \gamma E_G(q\zeta+\omega)\frac{R(t)}{R(t)+R^*}
$$

In [9]:
## python implementation
def genalgDE(Es, Eg, R, Rn, gamma, k, C, rho, ul, r, Rc):
    q = Es/Eg
    ps = Es/(Es+Eg)
    pg = Eg/(Es+Eg)
    zeta = ps*k/C
    beta = (q**2)*(1-k*ps)*ps + pg
    omega = beta/C
    h = R/(R+Rn)
    
    x = gamma*zeta*Es*h - 1 + rho*(Es**2)
    
    y = gamma*omega*Eg*h - ul + rho*(Eg**2)
    
    z = r*R*(1-R/Rc) - gamma*Eg*(q*zeta + omega)*h
    
    return x, y, z

In [10]:
@interact(k=(.05,1.5,.05), S0=(0,50,1), G0=(0,50,1), R0=(0,300,10), ul=(1,3,.1), gamma=(.01,5,.01), Rn=(0,2000,10), Rc=(0,2000), tsteps=(0,10000,100))
def genalgsim(S0=15, G0=15, R0=50, Rn=100, gamma=1.5, k=.35, C=100, rho=.001, ul=1.5, r=.2, Rc=200, tsteps=10000):
    
    dtau = .01
    x = np.empty([tsteps + 1,3])
    x[0] = [S0/C,G0/C,R0]
    for i in range(tsteps):
        s_dot, g_dot, R_dot = genalgDE(x[i][0], x[i][1], x[i][2], Rn, gamma, k, C, rho, ul, r, Rc)
        x[i+1][0] = x[i][0]+s_dot*dtau
        x[i+1][1] = x[i][1]+g_dot*dtau
        x[i+1][2] = x[i][2]+R_dot*dtau
    
    g = [y[1] for y in x]
    s = [y[0] for y in x]
    r = [y[2] for y in x]
    plt.plot(range(tsteps+1), g, c='b', label='gregarizing locusts\' energy')
    plt.plot(range(tsteps+1), s, c='r', label='solitary locusts\' energy')
    plt.plot(range(tsteps+1), r, c='g', label='resources')
    legend=plt.legend()
    plt.xlabel('time')
    plt.show()

interactive(children=(IntSlider(value=15, description='S0', max=50), IntSlider(value=15, description='G0', max…